In [1]:
using PCT

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


## $x^{\dagger} A x$

In [16]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j)))
blaserize(f)

(A) -> 
    𝒫((x) -> 
        x⁺⋅A⋅x)

In [17]:
df = redux(vdiff(f), settings=symmetry_settings) |> blaserize 

(A) -> 
    (x) -> 
        2.0⋅A⋅x

## Trace of Product

In [18]:
f, _ = @pct begin
    tr = (X::CM) -> sum(i, X(i, i))
    matmul = (A::CM, B::CM) -> (i::N, j::N) -> sum(n, A(i, n) * B(n, j))
    pullback((A::CM, B::CM) -> tr(matmul(A, B) + matmul(B, A)))
end; f

let 
    tr = (X) -> 
        ∑((i), X(i, i))
    matmul = (A, B) -> 
        (i, j) -> 
            ∑((n), A(i, n)⋅B(n, j))
    𝒫((A, B) -> 
        tr((matmul(A, B)+matmul(B, A))))
end

In [19]:
eval_all(f)

𝒫((A, B) -> 
    ∑((i), (∑((n), A(n, i)⋅B(i, n))+∑((n), A(i, n)⋅B(n, i)))))

In [20]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)  |> blaserize

(A, B) -> 
    2.0⋅B⁺, 2.0⋅A⁺

In [43]:
codegen(df)

:((A, B)->begin
          #= /Users/likangbo/kaer_morhen/projects/QLD/PCT/src/codegen.jl:58 =#
          #= /Users/likangbo/kaer_morhen/projects/QLD/PCT/src/codegen.jl:60 =#
          [Expr[:(2.0 .* transpose(conj(B))), :(2.0 .* transpose(conj(A)))]...]
      end)

## Conjugate Gradient

In [21]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) + sum(i, x(i) * b(i))
    pullback((α::R, β::R) -> E((i::N) -> x(i) + α * (r(i) + β * p(i))))
end; blaserize(f)

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        𝒫((α, β) -> 
            E((α⋅(β⋅p+r)+x)))
    end

In [22]:
df = vdiff(f) |> blaserize 

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        (α, β) -> 
            ∇(E)((α⋅(β⋅p+r)+x))ᵀ⋅(β⋅p+r), α⋅∇(E)((α⋅(β⋅p+r)+x))ᵀ⋅p
    end

In [23]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize 

(A, r, p, b, x) -> 
    (α, β) -> 
        ((β⋅p+r)ᵀ⋅b+(α⋅(β⋅p+r)+x)ᵀ⋅A⋅(β⋅p+r)), α⋅(bᵀ⋅p+(α⋅(β⋅p+r)+x)ᵀ⋅A⋅p)

## Lagrangian Mechanics

In [24]:
_, ctx = @pct begin
    @space X begin
        type = (RF, ) -> RF
        linear = true
    end
end; 

In [25]:
f, _ = @pct _ ctx (L::RO, D::X) -> pullback((r::RF) -> sum((t::R), L(r(t), D(r)(t)))); f

(L, D) -> 
    𝒫((r) -> 
        ∑((t), L(r(t), D(r)(t))))

In [26]:
redux(vdiff(f); settings=custom_settings(:clench_sum=>true))

(L, D) -> 
    (r) -> 
        (Dᵀ((_d) -> 
            ∇(\mathbb{z}_{1} \to L\left(r\left(\mathbb{d}\right), \mathbb{z}_{1}\right))(D(r)(_d)))+(_d) -> 
            ∇(\mathbb{z}_{2} \to L\left(\mathbb{z}_{2}, D\left(r\right)\left(\mathbb{d}\right)\right))(r(_d)))

## Hartree Fock Gradient and Hessian Vector Product

In [2]:
_, ctx = @pct begin    
    @space T begin
        type = (I, I, I, I) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [3]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))

In [5]:
df = redux(vdiff(f); settings=symmetry_settings)

1 sum_dist
∑((i, j, p, q, r, s), (∑((_z_1), (∑((_z_2), ((_d, _d_1) -> 
    ∑((_z_3), δ(i, _d_1, δ(q, _d, _z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'⋅δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))))))+(_d, _d_1) -> 
    δ(j, _d_1, δ(s, _d, _z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'⋅δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))))))+(_d, _d_1) -> 
    δ(j, _d_1, δ(r, _d, _z_1⋅δ(_z_1, C(p, i)', 1)⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))))+(_d, _d_1) -> 
    δ(i, _d_1, δ(p, _d, C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))))
Add
(∑((i, j, p, q, r, s, _z_1), (∑((_z_2), ((_d, _d_1) -> 
    ∑((_z_3), δ(i, _d_1, δ(q, _d, _z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'⋅δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))))))+(_d, _d_1) -> 
    δ(j, _d_1, δ(s, _d, _z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'⋅δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))))))+(_d, _d_1) -> 
    δ(j, _d_1, δ(r, _d, _z_1⋅δ(_z_1, C(p, i)', 1)⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))))+(_d, _d_1) -> 
    ∑((i, j, p, q, r, s), δ(i, _d_1, δ(p, _d, C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q

9 contract_delta
(_d, _d_1) -> 
    (∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, i, p, q, r, s), δ(r, _d, _z_1⋅δ(_z_1, C(p, i)', 1)⋅J(p, q, r, s)⋅C(s, _d_1)⋅C(q, i)))+∑((i, j, p, q, r, s, _z_1, _z_2), δ(j, _d_1, δ(s, _d, _z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'⋅δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))))+∑((i, j, p, q, r, s, _z_1, _z_2, _z_3), δ(i, _d_1, δ(q, _d, _z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'⋅δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))))))
Map
(_d, _d_1) -> 
    (∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, i, p, q, s), _z_1⋅δ(_z_1, C(p, i)', 1)⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, j, p, q, r, s, _z_1, _z_2), δ(j, _d_1, δ(s, _d, _z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'⋅δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))))+∑((i, j, p, q, r, s, _z_1, _z_2, _z_3), δ(i, _d_1, δ(q, _d, _z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'⋅δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))))))
-->

10 swallow
(_d, _d_1) -> 
    (∑((j, q, r, s), C(r, j)'⋅J(_d

19 contract_delta
(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, p, q, r), J(p, q, r, _d)'⋅C(q, i)'⋅C(p, i)⋅C(r, _d_1))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, _z_2, _z_3, j, p, q, r, s), δ(q, _d, _z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'⋅δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, _d_1)', _z_1, 1))))))
Map
(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, p, q, r), J(p, q, r, _d)'⋅C(q, i)'⋅C(p, i)⋅C(r, _d_1))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, _z_2, _z_3, j, p, r, s), _z_2'⋅_z_3'⋅_z_1'⋅J(p, _d, r, s)'⋅δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, _d_1)', _z_1, 1)))))
-->

20 swallow
(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, p, q, r), J(p, q, r, _d)'⋅C(q, i)'⋅C(p, i)⋅C(r, _d_1))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, _z_2, _z_3, j, p, r, s), _z_2'⋅_z_3'⋅_z_1'⋅J(p, _d, r, s)'⋅δ(C(s, j), _z_3, 

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            4.0⋅∑((j, p, r, s), C(s, j)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j))

In [17]:
df |> get_body |> decompose |> pp |> simplify |> first

(C, _k) -> 
    (_a, _a_1) -> 
        (∑((_d, i, p, q), (J(_d, _a, q, p)'⋅C(p, i)'⋅4.0⋅_k(_d, _a_1)⋅C(q, i)))+∑((_d, _d_1, q, r), (J(_d, r, q, _a)'⋅C(r, _d_1)'⋅4.0⋅_k(_d, _d_1)⋅C(q, _a_1)))+∑((_d, _d_1, p, r), (4.0⋅J(_d, r, _a, p)⋅_k(_d, _d_1)⋅C(p, _a_1)⋅C(r, _d_1))))

## Foster Boys

In [25]:
f, _ = @pct (R::Her, R2::Her) -> pullback((C::CM) -> begin 
        r = (i::N) -> sum((p, q), R(p, q) * C(p, i)' * C(q, i))
        sum(i, sum((p, q), R2(p, q) * C(p, i)' * C(q, i))) - sum(i, r(i)' * r(i))
    end); blaserize(f)

(R, R2) -> 
    𝒫((C) -> 
        let 
            r = (i) -> 
                (C⁺⋅R⋅C)(i, i)
            (tr(C⋅C⁺⋅R2)(-1.0⋅r⁺⋅r))
        end)

In [27]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize

(R, R2) -> 
    (C) -> 
        (-4.0⋅((_d, _d_1) -> 
            ((R⋅C)(_d, _d_1)⋅(C⁺⋅R⋅C)(_d_1, _d_1)))+2.0⋅R2⋅C)

## Localized Wannier Functions

In [20]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (N, N, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> C
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (N, N, BZ) -> C
    end
end;

In [21]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::N, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), ρ(n, b)' * ρ(n, b))
    end); f

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), (U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))
            ∑((n, b), (ρ(n, b)'⋅ρ(n, b)))
        end)

In [22]:
# df = vdiff(eval_all(f); settings=custom_settings(:logging=>true; preset = symmetry_settings))
df = redux(vdiff(eval_all(f)); settings=symmetry_settings)

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            (4.0⋅∑((b, p, k, _p, q), (U(_p, _d_1, (b+k))'⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k))))

In [23]:
_, ctx = @pct begin
    @space Tau begin
        type=(R, RV) -> RM
    end
    
    @space Nu begin
        type=(RV, ) -> R
    end
        
    @space S begin
        type=(RV, )->RV
    end
end;

In [66]:
f, _ = @pct _ ctx (τ::Tau, ν::Nu, p::S) -> :vdiff(pullback((x::RV) -> τ(ν(x), p(x)))); f

(τ, ν, p) -> 
    vdiff̂(𝒫((x) -> 
        τ(ν(x), p(x))))

In [15]:
df = process_directive(f)

(τ, ν, p) -> 
    (x) -> 
        (𝒫(ν)(x, 𝒫((_z) -> 
            τ(_z, p(x)))(ν(x), 1))+𝒫(p)(x, 𝒫((_z_1) -> 
            τ(ν(x), _z_1))(p(x), 1)))

In [2]:
_, ctx = @pct begin
    @space G begin
        type=(RV,)->R
    end
    end;

In [3]:
f, _ = @pct _ ctx (wstar::RV) -> (g::G) -> pullback((w::RV)  -> (g(w) - g(wstar))^2)
f

(wstar) -> 
    (g) -> 
        𝒫((w) -> 
            ((-1.0⋅g(wstar)+g(w)))^(2))

In [4]:
df = vdiff(f)

(wstar) -> 
    (g) -> 
        (w) -> 
            𝒫(g)(w, 2.0⋅(-1.0⋅g(wstar)+g(w)))

In [18]:
hf = df |> get_body |> get_body |> decompose |> pp

(w, _k) -> 
    (∑((_z), 𝒫((_z_2) -> 
        𝒫(g)(_z_2, _z))(w, δ(2.0⋅(-1.0⋅g(wstar)+g(w)), _z, _k)))+𝒫(g)(w, 2.0⋅(_z) -> 
        𝒫(g)(w, _z)⁺(_k)))

In [29]:
hf |> simplify |> first

(w, _k) -> 
    (𝒫(g)(w, 2.0⋅(_z) -> 
        𝒫(g)(w, _z)ᵀ(_k))+𝒫((_z_2) -> 
        𝒫(g)(_z_2, 2.0⋅(-1.0⋅g(wstar)+g(w))))(w, _k))